In [71]:
import cv2 as cv
import numpy as np
import pandas as pd
#import util
import os
import random

In [72]:
import glob
import cv2
import numpy as np
import os
import PIL
from PIL import Image
def FrameCapture(path,skipfactor=3): 
    #clearing extracted frames
    files = glob.glob('extractedframes/*')
    for f in files:
        os.remove(f)
        
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
    fps = vidObj.get(cv2.CAP_PROP_FPS)
    print(fps)
    vidObj.set(cv2.CAP_PROP_FPS, 30)
    fps = vidObj.get(cv2.CAP_PROP_FPS)
    print(fps)
    # Used as counter variable 
    count = 0
    ind = 0
  
    # checks whether frames were extracted 
    success = 1
    listOfImages = []
    while success: 
  
        # vidObj object calls read 
        # function extract frames 
        success, image = vidObj.read()
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Saves the frames with frame-count 
        if(count%skipfactor==0):
            listOfImages.append(image)
            
            #cv2.imwrite("extractedframes/frame%d.jpg" %ind, image)
            #cv2.imshow('frame%d.jpg'%ind,image)
            
            ind += 1
        count += 1
        type(image)
    return listOfImages

In [73]:
def ListGrayscaleFrameArrays(listOfFrames):
    listOfFrameArray=[]
    listOfFrames.pop()
    
    for ind,frame  in enumerate(listOfFrames):
        #converts RBG image to Gray scale and returns as numpy 2D array
        ff= frame.copy()
        gray = cv2.cvtColor(ff, cv2.COLOR_BGR2GRAY)
        
#         print(gray.shape)
#         scale_percent = 60 # percent of original size
#         width = int(1000)
#         height = int(1000)
#         dim = (width, height)
#         # resize image
#         resized = cv2.resize(gray, dim, interpolation = cv2.INTER_AREA)
        cv2.imwrite("grayframes/%d.jpg" %ind, gray)
        #using PIL
        #grayscalearray = np.array(Image.fromarray(frame).convert('L'))
        grayscalearray = np.array(gray)
        
        listOfFrameArray.append(grayscalearray)
        
    return listOfFrameArray



In [74]:

def StoreFrames(path,skipfactor=2):
    #extracted features stores all gray frames of a  video individually for further processing
    #clear extracted frames folder first
    files = glob.glob('extractedframes/*')
    for f in files:
        os.remove(f)
    FrameCapture(path,skipfactor)




In [75]:
#A  video is converted to a numpy array
def ConvertVideoToGrayscaleArray(path):
    listofImages = FrameCapture(path,2)

    GrayScaleList= ListGrayscaleFrameArrays(listofImages)
    
    ArrayofGrayScaleArray = np.array(GrayScaleList)
    
    #print(ArrayofGrayScaleArray.shape)
    return ArrayofGrayScaleArray

In [76]:
#movementfiltering

#stores binarized list of frames in jpg format 
def BinarizeGrayscaleArray(ArrayofGrayScaleArray,path,threshold=120):
    files = glob.glob('binaryframes/*')
    for f in files:
        os.remove(f)
    for i in range(1,ArrayofGrayScaleArray.shape[0]):
        binaryImage=abs(ArrayofGrayScaleArray[i]-ArrayofGrayScaleArray[i-1])
        data = Image.fromarray(binaryImage)
        data.save('temporalderivatives/'+'frame%d.jpg'%i)
        #Binarization
        binaryImage[binaryImage > threshold] = 255
        binaryImage[binaryImage <= threshold] = 0  
        data = Image.fromarray(binaryImage) 
        data.save(path+'/'+'frame%d.jpg'%i)
  

In [77]:
pwd

'C:\\Users\\DELL\\ViolenceDetection'

In [78]:
def erodeBinaryFrames(binaryframespath,erodedframespath):
    erosionkernel = np.array([
        [0,0,0,1,1,0,0,0],
        [0,0,1,1,1,1,0,0],
        [0,1,1,1,1,1,1,0],
        [1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1],
        [0,1,1,1,1,1,1,0],
        [0,0,1,1,1,1,0,0],
        [0,0,0,1,1,0,0,0]
    ],np.uint8)
    
    dilationkernel = np.array([
        [0,0,0,1,1,0,0,0],
        [0,0,1,1,1,1,0,0],
        [0,1,1,1,1,1,1,0],
        [1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1],
        [0,1,1,1,1,1,1,0],
        [0,0,1,1,1,1,0,0],
        [0,0,0,1,1,0,0,0]
    ],np.uint8)
    

    # Taking a matrix of size 5 as the kernel 
    kernel = np.ones((3,3), np.uint8)
    kernel[0][0]=0
    kernel[0][2]=0
    kernel[2][0]=0
    kernel[2][2]=0
    print(kernel)
    # The first parameter is the original image, 
    # kernel is the matrix with which image is  
    # convolved and third parameter is the number  
    # of iterations, which will determine how much  
    # you want to erode/dilate a given image.
    files = glob.glob(erodedframespath+'/*')
    for f in files:
        os.remove(f)
    files = glob.glob(binaryframespath+'/*')
    for ind,f in enumerate(files):
        img = cv2.imread(f, 0) 
        img_erosion = cv2.erode(img, erosionkernel, iterations=1) 
        img_dilation = cv2.dilate(img_erosion, kernel, iterations=1)
        cv2.imwrite(erodedframespath+"/frame%d.jpg" %ind,img_dilation)



In [79]:
def preprocess(videopath):
    #A  video is converted to a numpy array
    ArrayofGrayScaleArray= ConvertVideoToGrayscaleArray(videopath)
    BinarizeGrayscaleArray(ArrayofGrayScaleArray,'binaryframes')
    erodeBinaryFrames("binaryframes","morphologicallyopenedframes")
#upto morphological opening
#preProcess('samplefights/fi1_xvid.avi')

In [80]:
import math

cells = [15, 31, 47, 63, 79, 95, 111, 127, 143, 159, 175, 191, 207, 223, 239, 255]


def gen_neighbors(x, y):
    neighbor_1 = [[x - 8, y + 8], [x - 7, y + 8], [x - 6, y + 8], [x - 5, y + 8],
          [x - 8, y + 7], [x - 7, y + 7], [x - 6, y + 7], [x - 5, y + 7],
          [x - 8, y + 6], [x - 7, y + 6], [x - 6, y + 6], [x - 5, y + 6],
          [x - 8, y + 5], [x - 7, y + 5], [x - 6, y + 5], [x - 5, y + 5]]

    neighbor_2 = [[x - 4, y + 8], [x - 3, y + 8], [x - 2, y + 8], [x - 1, y + 8],
          [x - 4, y + 7], [x - 3, y + 7], [x - 2, y + 7], [x - 1, y + 7],
          [x - 4, y + 6], [x - 3, y + 6], [x - 2, y + 6], [x - 1, y + 6],
          [x - 4, y + 5], [x - 3, y + 5], [x - 2, y + 5], [x - 1, y + 5]]

    neighbor_3 = [[x + 1, y + 8], [x + 2, y + 8], [x + 3, y + 8], [x + 4, y + 8],
          [x + 1, y + 7], [x + 2, y + 7], [x + 3, y + 7], [x + 4, y + 7],
          [x + 1, y + 6], [x + 2, y + 6], [x + 3, y + 6], [x + 4, y + 6],
          [x + 1, y + 5], [x + 2, y + 5], [x + 3, y + 5], [x +  4, y + 5]]

    neighbor_4 = [[x + 5, y + 8], [x + 6, y + 8], [x + 7, y + 8], [x + 8, y + 8],
          [x + 5, y + 7], [x + 6, y + 7], [x + 7, y + 7], [x + 8, y + 7],
          [x + 5, y + 6], [x + 6, y + 6], [x + 7, y + 6], [x + 8, y + 6],
          [x + 5, y + 5], [x + 6, y + 5], [x + 7, y + 5], [x + 8, y + 5]]

    neighbor_5 = [[x - 8, y + 4], [x - 7, y + 4], [x - 6, y + 4], [x - 5, y + 4],
          [x - 8, y + 3], [x - 7, y + 3], [x - 6, y + 3], [x - 5, y + 3],
          [x - 8, y + 2], [x - 7, y + 2], [x - 6, y + 2], [x - 5, y + 2],
          [x - 8, y + 1], [x - 7, y + 1], [x - 6, y + 1], [x - 5, y + 1]]

    neighbor_6 = [[x - 4, y + 4], [x - 3, y + 4], [x - 2, y + 4], [x - 1, y + 4],
          [x - 4, y + 3], [x - 3, y + 3], [x - 2, y + 3], [x - 1, y + 3],
          [x - 4, y + 2], [x - 3, y + 2], [x - 2, y + 2], [x - 1, y + 2],
          [x - 4, y + 1], [x - 3, y + 1], [x - 2, y + 1], [x - 1, y + 1]]

    neighbor_7 = [[x + 1, y + 4], [x + 2, y + 4], [x + 3, y + 4], [x + 4, y + 4],
          [x + 1, y + 3], [x + 2, y + 3], [x + 3, y + 3], [x + 4, y + 3],
          [x + 1, y + 2], [x + 2, y + 2], [x + 3, y + 2], [x + 4, y + 2],
          [x + 1, y + 1], [x + 2, y + 1], [x + 3, y + 1], [x + 4, y + 1]]

    neighbor_8 = [[x + 5, y + 4], [x + 6, y + 4], [x + 7, y + 4], [x + 8, y + 4],
          [x + 5, y + 3], [x + 6, y + 3], [x + 7, y + 3], [x + 8, y + 3],
          [x + 5, y + 2], [x + 6, y + 2], [x + 7, y + 2], [x + 8, y + 2],
          [x + 5, y + 1], [x + 6, y + 1], [x + 7, y + 1], [x + 8, y + 1]]

    neighbor_9 = [[x - 8, y - 4], [x - 7, y - 4], [x - 6, y - 4], [x - 5, y - 4],
          [x - 8, y - 3], [x - 7, y - 3], [x - 6, y - 3], [x - 5, y - 3],
          [x - 8, y - 2], [x - 7, y - 2], [x - 6, y - 2], [x - 5, y - 2],
          [x - 8, y - 1], [x - 7, y - 1], [x - 6, y - 1], [x - 5, y - 1]]

    neighbor_10 = [[x - 4, y - 4], [x - 3, y - 4], [x - 2, y - 4], [x - 1, y - 4],
           [x - 4, y - 3], [x - 3, y - 3], [x - 2, y - 3], [x - 1, y - 3],
           [x - 4, y - 2], [x - 3, y - 2], [x - 2, y - 2], [x - 1, y - 2],
           [x - 4, y - 1], [x - 3, y - 1], [x - 2, y - 1], [x - 1, y - 1]]

    neighbor_11 = [[x + 1, y - 4], [x + 2, y - 4], [x + 3, y - 4], [x + 4, y - 4],
           [x + 1, y - 3], [x + 2, y - 3], [x + 3, y - 3], [x + 4, y - 3],
           [x + 1, y - 2], [x + 2, y - 2], [x + 3, y - 2], [x + 4, y - 2],
           [x + 1, y - 1], [x + 2, y - 1], [x + 3, y - 1], [x + 4, y - 1]]

    neighbor_12 = [[x + 5, y - 4], [x + 6, y - 4], [x + 7, y - 4], [x + 8, y - 4],
           [x + 5, y - 3], [x + 6, y - 3], [x + 7, y - 3], [x + 8, y - 3],
           [x + 5, y - 2], [x + 6, y - 2], [x + 7, y - 2], [x + 8, y - 2],
           [x + 5, y - 1], [x + 6, y - 1], [x + 7, y - 1], [x + 8, y - 1]]

    neighbor_13 = [[x - 8, y - 8], [x - 7, y - 8], [x - 6, y - 8], [x - 5, y - 8],
           [x - 8, y - 7], [x - 7, y - 7], [x - 6, y - 7], [x - 5, y - 7],
           [x - 8, y - 6], [x - 7, y - 6], [x - 6, y - 6], [x - 5, y - 6],
           [x - 8, y - 5], [x - 7, y - 5], [x - 6, y - 5], [x - 5, y - 5]]

    neighbor_14 = [[x - 4, y - 8], [x - 3, y - 8], [x - 2, y - 8], [x - 1, y - 8],
           [x - 4, y - 7], [x - 3, y - 7], [x - 2, y - 7], [x - 1, y - 7],
           [x - 4, y - 6], [x - 3, y - 6], [x - 2, y - 6], [x - 1, y - 6],
           [x - 4, y - 5], [x - 3, y - 5], [x - 2, y - 5], [x - 1, y - 5]]

    neighbor_15 = [[x + 1, y - 8], [x + 2, y - 8], [x + 3, y - 8], [x + 4, y - 8],
           [x + 1, y - 7], [x + 2, y - 7], [x + 3, y - 7], [x + 4, y - 7],
           [x + 1, y - 6], [x + 2, y - 6], [x + 3, y - 6], [x + 4, y - 6],
           [x + 1, y - 5], [x + 2, y - 5], [x + 3, y - 5], [x + 4, y - 5]]

    neighbor_16 = [[x + 5, y - 8], [x + 6, y - 8], [x + 7, y - 8], [x + 8, y - 8],
           [x + 5, y - 7], [x + 6, y - 7], [x + 7, y - 7], [x + 8, y - 7],
           [x + 5, y - 6], [x + 6, y - 6], [x + 7, y - 6], [x + 8, y - 6],
           [x + 5, y - 5], [x + 6, y - 5], [x + 7, y - 5], [x + 8, y - 5]]

    neighbors = neighbor_1+neighbor_2+neighbor_3+neighbor_4+neighbor_5+neighbor_6+neighbor_7+neighbor_8+ \
                neighbor_9+neighbor_10+neighbor_11+neighbor_12+neighbor_13+neighbor_14+neighbor_15+neighbor_16

    return neighbors

def gen_arc_histogram(direction, histogram):
    if direction > (2*math.pi)/3 and direction <= math.pi/3:
        histogram[0]+=1
    elif direction > math.pi/3 and direction <= math.pi/6:
        histogram[1]+=1
    elif direction > math.pi/6 and direction <= (11*math.pi)/6:
        histogram[2]+=1
    elif direction > (11*math.pi)/6 and direction <= (5*math.pi)/3:
        histogram[3]+=1
    elif direction > (5*math.pi)/3 and direction <= (4*math.pi)/3:
        histogram[4]+=1
    elif direction > (4*math.pi)/3 and direction <= (5*math.pi)/4:
        histogram[5]+=1
    elif direction > (5*math.pi)/4 and direction <= math.pi:
        histogram[6]+=1
    elif direction > math.pi and direction <= (2*math.pi)/3:
        histogram[7]+=1
    return histogram

In [81]:
#SIFT

import cv2 as cv
import numpy as np
import pandas as pd
#import util
import os
import random

# params for ShimTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# params for Lucas Kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))


def capture_frame(video_path, i):
    cap = cv.VideoCapture(video_path)
    cap.set(1, i)
    ret, frame = cap.read()

    return ret, frame


def count_frames(video_path):
    cap = cv.VideoCapture(video_path)
    frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

    return frames


def gen_sift_features(frame):
    sift = cv.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(frame, None)

    return keypoints, descriptors


def keypoints_to_coordinates(keypoints):
    keypoints_xy = []

    for kp in keypoints:
        keypoints_xy.append([kp.pt[0], kp.pt[1]])

    return keypoints_xy


def has_sufficient_motion(keypoints_xy, keypoints, descriptors, p1, lambd):
    sm_keypoints_xy = []
    sm_keypoints = []
    sm_descriptors = []

    for i in range(len(keypoints)):
        distance_x = keypoints_xy[i].T[0] - p1[i].T[0]
        distance_y = keypoints_xy[i].T[1] - p1[i].T[1]

        if distance_x > lambd or distance_y > lambd:
            sm_keypoints_xy.append([int(keypoints_xy[i].T[0]), int(keypoints_xy[i].T[1])])
            sm_keypoints.append(keypoints[i])
            sm_descriptors.append(descriptors[i])

    return sm_keypoints_xy, sm_keypoints, sm_descriptors


def gen_hof(x, y, frame, next_frame):
    hof = []
    histogram = [0, 0, 0, 0, 0, 0, 0, 0]
    neighbors = gen_neighbors(x, y)
    neighbors = np.float32(np.array(neighbors)[:, np.newaxis, :])
    p1, st, err = cv.calcOpticalFlowPyrLK(frame, next_frame, neighbors, None, **lk_params)

    for i in range(len(p1)):
        direction = np.arctan(p1[i].T[0]/p1[i].T[1])
        histogram = gen_arc_histogram(direction, histogram)
        if i in cells:
            hof += histogram
            histogram = [0, 0, 0, 0, 0, 0, 0, 0]

    return hof




In [82]:
# def gen_mosift_features(video_path, lambd, interval, sample_size):
#     frames = count_frames(video_path)
#     mosift_descriptors = []

#     for i in range(1, frames-1, interval):
#         #try:
#         ret, frame = capture_frame(video_path, i)
#         ret, next_frame = capture_frame(video_path, i+1)
#         print(type(frame))
#         keypoints, descriptors = gen_sift_features(frame)
#         keypoints_xy = keypoints_to_coordinates(keypoints)
#         keypoints_xy = np.float32(np.array(keypoints_xy)[:, np.newaxis, :])
#         p1, st, err = cv.calcOpticalFlowPyrLK(frame, next_frame, keypoints_xy, None, **lk_params)
#         sm_keypoints_xy, sm_keypoints, sm_descriptors = has_sufficient_motion(keypoints_xy, keypoints, descriptors, p1, lambd)

#         for j in range(len(sm_keypoints)):
#             hof = np.array(gen_hof(sm_keypoints_xy[j][0], sm_keypoints_xy[j][1], frame, next_frame))
#             mosift_descriptors.append(list(np.concatenate((sm_descriptors[j], hof))))
#         #except:
#             #print('Exception in '+video_path+', frame '+str(i))

#     random_mosift_descriptors = random.sample(mosift_descriptors, k=int(len(mosift_descriptors)*sample_size))

#     return random_mosift_descriptors


# def run_feature_extractor(input_path, output_path, lambd, interval, sample_size):
#     listing = os.listdir(input_path)
#     progress_count = 0
#     for video_name in listing:
#         progress_count += 1
#         print("# progress: "+str(progress_count)+'/'+str(len(listing)))
#         video_path = input_path+'/'+ video_name
        
#         df_mosift_features = pd.DataFrame(gen_mosift_features(video_path, lambd, interval, sample_size))
#         print(df_mosift_features.shape)
#         #df_mosift_features.to_csv(output_path+'/'+ video_name[:-4]+".csv", mode='a', header=False, index=False)

# run_feature_extractor('siftvideo','siftoutput',0.9,1,1)

In [92]:
def gen_mosift_features(morphopath, lambd, interval, sample_size):
    files = glob.glob(morphopath+'/*')
    frames=0
    for f in files:
        frames = frames+1
    mosift_descriptors = []

    for i in range(1, frames-1, interval):
        #try:
        j=i+1
        print(i)
        #frame = cv2.imread(morphopath+'/frame%d'%i,cv2.IMREAD_UNCHANGED)
        frame = cv.imread(morphopath+'/frame%d.jpg'%i,1)
        frame=cv.normalize(frame, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
        next_frame = cv.imread(morphopath+'/frame%d.jpg'%j,1)
        next_frame=cv.normalize(next_frame, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = gen_sift_features(frame)
        keypoints_xy = keypoints_to_coordinates(keypoints)
        keypoints_xy = np.float32(np.array(keypoints_xy)[:, np.newaxis, :])
        p1, st, err = cv.calcOpticalFlowPyrLK(frame, next_frame, keypoints_xy, None, **lk_params)
        sm_keypoints_xy, sm_keypoints, sm_descriptors = has_sufficient_motion(keypoints_xy, keypoints, descriptors, p1, lambd)

        for j in range(len(sm_keypoints)):
            hof = np.array(gen_hof(sm_keypoints_xy[j][0], sm_keypoints_xy[j][1], frame, next_frame))
            mosift_descriptors.append(list(np.concatenate((sm_descriptors[j], hof))))
        #except:
            #print('Exception in '+video_path+', frame '+str(i))

    random_mosift_descriptors = random.sample(mosift_descriptors, k=int(len(mosift_descriptors)*sample_size))

    return random_mosift_descriptors


def run_feature_extractor(morphopath, output_path,video_name, lambd, interval, sample_size):
    
    df_mosift_features = pd.DataFrame(gen_mosift_features(morphopath, lambd, interval, sample_size))
    print(df_mosift_features)
    df_mosift_features.to_csv(output_path+'/'+video_name[13:-4]+".csv", mode='a', header=False, index=False)

#run_feature_extractor('morphologicallyopenedframes','siftoutput',video_name,0.85,1,1)

In [93]:
pwd

'C:\\Users\\DELL\\ViolenceDetection'

In [106]:
#final upto extracting descriptors and stroing them in "siftoutput" folder as .csv files.
def storeDescriptors(videopath):
    videos = glob.glob(videopath+'/*')
    for video in videos:
        #video=video[13:]
        print(video)
        preprocess(video)
        run_feature_extractor('morphologicallyopenedframes','siftoutput',video,0.85,1,1)
storeDescriptors('samplefights')

samplefights\fi1_xvid.avi
25.0
25.0
[[0 1 0]
 [1 1 1]
 [0 1 0]]
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
       0     1    2     3     4     5     6     7      8      9    ...  246  \
0      0.0   0.0  0.0   0.0   0.0   0.0   0.0   2.0    0.0    0.0  ...  0.0   
1      7.0   0.0  0.0  46.0  72.0  24.0  23.0  33.0   28.0    0.0  ...  0.0   
2     32.0   2.0  0.0   0.0   0.0   0.0   0.0   3.0  137.0   19.0  ...  0.0   
3      6.0  11.0  0.0   0.0   0.0   0.0   0.0   0.0   75.0  127.0  ...  0.0   
4      0.0   0.0  0.0   0.0   0.0   0.0   0.0   0.0    0.0    0.0  ...  0.0   
...    ...   ...  ...   ...   ...   ...   ...   ...    ...    ...  ...  ...   
3808  56.0   6.0  0.0   1.0   0.0   0.0   0.0   1.0  152.0   52.0  ...  0.0   
3809   1.0   0.0  0.0   0.0   2.0  11.0  26.0  25.0    5.0    3.0  ...  0.0   
3810   0.0   0.0  0.0   0.0   0.0   0.0   1.0   2.0    0.0    0.0  ...  0.0   
3811   8.0  31.0  0.0   0.0   0.0   0.0   0.0   0.0  129.0  142.0  ...  0.0   
3812   0.0   0.0  0.0   0

In [107]:
from sklearn.cluster import KMeans
def bagOfVisualWords(descriptorspath,noofclusters):
    descriptors = glob.glob(descriptorspath+'/*')
    maindataframe=pd.DataFrame()
    for descriptor in descriptors:
        df=pd.read_csv(descriptor,header=None)
        maindataframe=maindataframe.append(df)
        print(df.shape)
    print(maindataframe.shape)
    
    kmeans = KMeans(n_clusters = noofclusters).fit(maindataframe)
    
    return kmeans
           
kmeans=bagOfVisualWords('siftoutput',200)
print(type(kmeans))



(3813, 256)
(29794, 256)
(13908, 256)
(47515, 256)
<class 'sklearn.cluster._kmeans.KMeans'>


In [144]:
            
numberofclusters=200 
def featureExtractor(descriptorspath,numberofclusters):
    descriptors = glob.glob(descriptorspath+'/*')
    trainfeatures=np.empty(shape=[0, 200])
    for descriptor in descriptors:
        
        df=pd.read_csv(descriptor,header=None)
        print(df.shape)
        histogram=kmeans.predict(df)
        
        featurevector=np.zeros(numberofclusters,)
        for i in range(0,len(histogram)):
            featurevector[histogram[i]]=featurevector[histogram[i]]+1
        print(featurevector)
        trainfeatures=np.vstack([trainfeatures, featurevector])    
    
    return trainfeatures


    
    

    


In [145]:
trainset=featureExtractor('siftoutput',200)

(3813, 256)
[21. 13. 13. 16. 23. 89. 11.  9. 27. 15. 60. 17. 12. 22. 29. 26. 39. 44.
 15.  1. 13. 12. 13. 11. 13. 13. 14. 21. 16. 13.  7. 14. 29. 17. 16.  8.
 15. 20.  9. 27. 11. 19. 23. 22. 22. 13. 33. 23. 10. 32. 15. 20. 13. 27.
 15. 14. 37.  9. 28. 25. 10. 13. 14. 10. 13. 13. 22. 19.  8. 16. 22. 20.
 14. 14. 11. 14. 42. 22.  4. 35. 22. 12. 19. 24. 48. 18. 21. 22. 13. 14.
 12. 43. 16. 12. 16.  9. 13. 20. 16. 13. 18. 18. 17. 26. 21.  7. 22. 20.
  7. 14. 13. 16. 13. 17. 25. 22. 14. 20. 16. 14. 23. 27. 19.  7. 22. 15.
 28. 23. 24. 16. 17. 23. 22. 21. 26. 18. 22. 19.  7.  9. 40. 18. 23. 33.
 38. 45. 50. 14. 19. 23. 13. 15. 13. 10. 40. 18. 21.  6. 20. 18. 20. 14.
 22. 22. 14. 13. 17. 15. 19. 13. 15. 21.  6. 23. 14. 20. 11. 13. 14.  4.
 14. 17. 26. 30. 20. 12. 29. 14. 22. 17. 21.  7.  9. 19. 11. 10. 13. 28.
 16. 30.]
(29794, 256)
[172. 145. 126. 127. 121. 867. 102. 124. 181. 166. 235.  95.  81. 142.
 187. 192. 233. 223. 112.  77. 157. 161. 134. 139.  97. 143.  95. 167.
 146. 143. 100. 164.

In [135]:
#print(trainset)

[[21. 13. 13. 16. 23. 89. 11.  9. 27. 15. 60. 17. 12. 22. 29. 26. 39. 44.
  15.  1. 13. 12. 13. 11. 13. 13. 14. 21. 16. 13.  7. 14. 29. 17. 16.  8.
  15. 20.  9. 27. 11. 19. 23. 22. 22. 13. 33. 23. 10. 32. 15. 20. 13. 27.
  15. 14. 37.  9. 28. 25. 10. 13. 14. 10. 13. 13. 22. 19.  8. 16. 22. 20.
  14. 14. 11. 14. 42. 22.  4. 35. 22. 12. 19. 24. 48. 18. 21. 22. 13. 14.
  12. 43. 16. 12. 16.  9. 13. 20. 16. 13. 18. 18. 17. 26. 21.  7. 22. 20.
   7. 14. 13. 16. 13. 17. 25. 22. 14. 20. 16. 14. 23. 27. 19.  7. 22. 15.
  28. 23. 24. 16. 17. 23. 22. 21. 26. 18. 22. 19.  7.  9. 40. 18. 23. 33.
  38. 45. 50. 14. 19. 23. 13. 15. 13. 10. 40. 18. 21.  6. 20. 18. 20. 14.
  22. 22. 14. 13. 17. 15. 19. 13. 15. 21.  6. 23. 14. 20. 11. 13. 14.  4.
  14. 17. 26. 30. 20. 12. 29. 14. 22. 17. 21.  7.  9. 19. 11. 10. 13. 28.
  16. 30.]
 [21. 13. 13. 16. 23. 89. 11.  9. 27. 15. 60. 17. 12. 22. 29. 26. 39. 44.
  15.  1. 13. 12. 13. 11. 13. 13. 14. 21. 16. 13.  7. 14. 29. 17. 16.  8.
  15. 20.  9. 27. 11. 19. 2